## Prerequisites

Before running the first cell, make sure you're authenticated with Azure CLI. Run this command in your terminal:

```bash
az login
```

or

```bash
az login --use-device-code
```

# Azure AI Agent with Existing Thread Example

This notebook demonstrates working with pre-existing conversation threads by providing thread IDs for thread reuse patterns.

## Features Covered:
- Creating and managing persistent conversation threads
- Working with existing thread IDs
- Thread lifecycle management (create, use, delete)
- Conversation continuity across sessions
- Thread initialization and validation

## Prerequisites

Before running this notebook, ensure you have:

1. **Azure AI Project**: Access to an Azure AI Foundry project with deployed models
2. **Authentication**: Azure CLI installed and authenticated (`az login --use-device-code`)
3. **Environment Variables**: Set up your `.env` file with connection details
4. **Dependencies**: Required agent-framework packages installed

If you need to use a different tenant, specify the tenant ID:
```bash
az login --tenant <tenant-id>
```

This example demonstrates how to work with existing threads to maintain conversation continuity.

## Import Libraries

Import the required libraries for Azure AI agent functionality.

In [1]:
import os
from pathlib import Path
from azure.identity import AzureCliCredential, InteractiveBrowserCredential
from dotenv import load_dotenv  # For loading environment variables from .env file
import asyncio
from random import randint
from typing import Annotated
from agent_framework import ChatAgent
from agent_framework.azure import AzureAIAgentClient
from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import AzureCliCredential
from pydantic import Field

# Get the path to the .env file which is in the parent directory
notebook_path = Path().absolute()  # Get absolute path of current notebook
parent_dir = notebook_path.parent  # Get parent directory
load_dotenv('../../.env')  # Load environment variables from .env file

False

## Check Environment Variables

Let's verify that the required environment variables are set:

In [2]:
# Check required environment variables
endpoint = os.getenv('AZURE_AI_PROJECT_ENDPOINT')
if endpoint:
    print(f"✅ AZURE_AI_PROJECT_ENDPOINT: {endpoint}")
else:
    print("❌ AZURE_AI_PROJECT_ENDPOINT: Not set")
    print("⚠️  Please set the AZURE_AI_PROJECT_ENDPOINT environment variable")

✅ AZURE_AI_PROJECT_ENDPOINT: https://demopocaifoundry.services.ai.azure.com/api/projects/demoproject


## Define Function Tools

Let's define a simple weather function that our agent can use:

In [3]:
def get_weather(
    location: Annotated[str, Field(description="The location to get the weather for.")],
) -> str:
    """Get the weather for a given location."""
    conditions = ["sunny", "cloudy", "rainy", "stormy"]
    return f"The weather in {location} is {conditions[randint(0, 3)]} with a high of {randint(10, 30)}°C."

## Create and Use Existing Thread

This example shows how to:
1. Create a thread that persists beyond the current session
2. Use the thread ID to continue conversations
3. Properly clean up thread resources

In [4]:
async def main() -> None:
    print("=== Azure AI Chat Client with Existing Thread ===")

    # Create the client
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):
        # Create a thread that will persist
        created_thread = await client.agents.threads.create()
        print(f"✅ Created thread with ID: {created_thread.id}")

        try:
            async with ChatAgent(
                # passing in the client is optional here, so if you take the agent_id from the portal
                # you can use it directly without the two lines above.
                chat_client=AzureAIAgentClient(project_client=client, async_credential=credential),
                instructions="You are a helpful weather agent.",
                tools=get_weather,
            ) as agent:
                thread = agent.get_new_thread(service_thread_id=created_thread.id)
                assert thread.is_initialized
                print(f"✅ Thread is initialized: {thread.is_initialized}")
                
                result = await agent.run("What's the weather like in Tokyo?", thread=thread)
                print(f"Result: {result}\n")
        finally:
            # Clean up the thread manually
            await client.agents.threads.delete(created_thread.id)
            print(f"🗑️  Deleted thread with ID: {created_thread.id}")


## Execute the Example

Run the main function to see the existing thread workflow:

In [5]:
# Run the main function
await main()

=== Azure AI Chat Client with Existing Thread ===


[2026-02-12 11:52:10 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


✅ Created thread with ID: thread_LyxJzOPenVdozx5g3sKeKEee
✅ Thread is initialized: True


[2026-02-12 11:52:13 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


Result: The weather in Tokyo is currently stormy with a high temperature of 25°C. Stay cautious and prepared for stormy conditions!

🗑️  Deleted thread with ID: thread_LyxJzOPenVdozx5g3sKeKEee


## Multi-Turn Conversation Example

This example shows how to have a multi-turn conversation using a persistent thread:

In [6]:
async def multi_turn_conversation_example():
    """Example showing multi-turn conversation with persistent thread."""
    print("=== Multi-Turn Conversation Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):
        # Create a persistent thread for the conversation
        conversation_thread = await client.agents.threads.create()
        print(f"📝 Started conversation with thread ID: {conversation_thread.id}")
        
        try:
            async with ChatAgent(
                chat_client=AzureAIAgentClient(project_client=client, async_credential=credential),
                instructions="You are a helpful weather agent. Remember previous conversations and provide contextual responses.",
                tools=get_weather,
            ) as agent:
                
                # Get the thread object
                thread = agent.get_new_thread(service_thread_id=conversation_thread.id)
                
                # Multi-turn conversation
                conversation = [
                    "What's the weather like in Paris?",
                    "How about in London?",
                    "Which city has better weather between the two I just asked about?",
                    "Thank you for the information about Paris and London."
                ]
                
                for i, user_message in enumerate(conversation, 1):
                    print(f"\n--- Turn {i} ---")
                    print(f"🤔 User: {user_message}")
                    
                    result = await agent.run(user_message, thread=thread)
                    print(f"🤖 Agent: {result.text}")
                    
        finally:
            # Clean up the thread
            await client.agents.threads.delete(conversation_thread.id)
            print(f"\n🗑️  Conversation ended. Deleted thread: {conversation_thread.id}")

# Run the multi-turn conversation example
await multi_turn_conversation_example()

=== Multi-Turn Conversation Example ===


[2026-02-12 11:52:43 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


📝 Started conversation with thread ID: thread_igU8m87q7bmkgNgDykMCcrup

--- Turn 1 ---
🤔 User: What's the weather like in Paris?


[2026-02-12 11:52:46 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.
[2026-02-12 11:52:50 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


🤖 Agent: The weather in Paris is sunny, with a high temperature of 30°C.

--- Turn 2 ---
🤔 User: How about in London?


[2026-02-12 11:52:53 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.
[2026-02-12 11:52:55 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


🤖 Agent: The weather in London is sunny, with a high of 23°C.

--- Turn 3 ---
🤔 User: Which city has better weather between the two I just asked about?


[2026-02-12 11:52:59 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


🤖 Agent: Both cities have sunny weather today, but Paris is warmer with a high of 30°C compared to London's 23°C. If you prefer warmer temperatures, Paris has the better weather!

--- Turn 4 ---
🤔 User: Thank you for the information about Paris and London.
🤖 Agent: You're very welcome! Let me know if you need more weather updates or have any other questions. Have a great day! ☀️

🗑️  Conversation ended. Deleted thread: thread_igU8m87q7bmkgNgDykMCcrup


## Thread Management Example

This example demonstrates advanced thread management operations:

In [7]:
async def thread_management_example():
    """Advanced example showing thread management operations."""
    print("=== Thread Management Example ===")
    
    async with (
        AzureCliCredential() as credential,
        AIProjectClient(endpoint=os.environ["AZURE_AI_PROJECT_ENDPOINT"], credential=credential) as client,
    ):
        # Create multiple threads
        threads = []
        for i in range(3):
            thread = await client.agents.threads.create()
            threads.append(thread)
            print(f"📝 Created thread {i+1}: {thread.id}")
        
        try:
            async with ChatAgent(
                chat_client=AzureAIAgentClient(project_client=client, async_credential=credential),
                instructions="You are a helpful assistant. Each thread represents a different conversation.",
                tools=get_weather,
            ) as agent:
                
                # Use each thread for a different topic
                topics = [
                    ("Weather in New York", "What's the weather in New York?"),
                    ("Weather in Berlin", "How's the weather in Berlin?"),
                    ("Weather comparison", "Compare weather in Asia vs Europe")
                ]
                
                for i, (topic, query) in enumerate(topics):
                    print(f"\n--- Thread {i+1}: {topic} ---")
                    thread_obj = agent.get_new_thread(service_thread_id=threads[i].id)
                    
                    print(f"🤔 User: {query}")
                    result = await agent.run(query, thread=thread_obj)
                    print(f"🤖 Agent: {result.text}")
                    
                # Follow up on first thread to show conversation continuity
                print("\n--- Following up on Thread 1 ---")
                thread_1 = agent.get_new_thread(service_thread_id=threads[0].id)
                followup = "Is it good weather for outdoor activities?"
                print(f"🤔 User: {followup}")
                result = await agent.run(followup, thread=thread_1)
                print(f"🤖 Agent: {result.text}")
                
        finally:
            # Clean up all threads
            for i, thread in enumerate(threads):
                await client.agents.threads.delete(thread.id)
                print(f"🗑️  Deleted thread {i+1}: {thread.id}")

# Run the thread management example
await thread_management_example()

=== Thread Management Example ===
📝 Created thread 1: thread_KDCHPPJXAlh8XJjF0X3bBTiR
📝 Created thread 2: thread_t6qNrFa2bTFZv2m2DBB9hd5u


[2026-02-12 11:53:22 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


📝 Created thread 3: thread_RjSkqoPMs6ogw9QVmtsZeU3F

--- Thread 1: Weather in New York ---
🤔 User: What's the weather in New York?


[2026-02-12 11:53:25 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.
[2026-02-12 11:53:27 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


🤖 Agent: The weather in New York is rainy with a high of 11°C.

--- Thread 2: Weather in Berlin ---
🤔 User: How's the weather in Berlin?


[2026-02-12 11:53:30 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.
[2026-02-12 11:53:32 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


🤖 Agent: The weather in Berlin is sunny with a high of 26°C.

--- Thread 3: Weather comparison ---
🤔 User: Compare weather in Asia vs Europe


[2026-02-12 11:53:35 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.
[2026-02-12 11:53:38 - c:\src\ai-foundry-e2e-lab\.venv\Lib\site-packages\agent_framework\_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


🤖 Agent: Currently, in Asia, the weather is stormy with a high of 26°C, while in Europe, it is rainy with a high of 17°C. It appears Asia is warmer but experiencing stormier conditions compared to Europe.

--- Following up on Thread 1 ---
🤔 User: Is it good weather for outdoor activities?
🤖 Agent: Since the weather is rainy in New York and has a high of 11°C, it might not be the best for outdoor activities. Rain can make conditions wet and slippery, and cooler temperatures may not be ideal for comfort. If you're planning to be outdoors, remember to dress warmly and bring rain gear. Alternatively, it might be a good day for indoor activities!
🗑️  Deleted thread 1: thread_KDCHPPJXAlh8XJjF0X3bBTiR
🗑️  Deleted thread 2: thread_t6qNrFa2bTFZv2m2DBB9hd5u
🗑️  Deleted thread 3: thread_RjSkqoPMs6ogw9QVmtsZeU3F


## Key Takeaways

1. **Persistent Threads**: Threads can persist beyond a single session, enabling conversation continuity
2. **Thread Initialization**: Always verify that threads are properly initialized before use
3. **Conversation Context**: Threads maintain conversation history, allowing for contextual responses
4. **Resource Management**: Always clean up threads when they're no longer needed
5. **Multi-Threading**: You can manage multiple separate conversations using different thread IDs
6. **Thread Reuse**: Threads can be reused across different agent instances

## Best Practices

1. **Thread Lifecycle**: Create threads when starting conversations and delete them when finished
2. **Error Handling**: Use try-finally blocks to ensure thread cleanup
3. **Thread Validation**: Always check if threads are initialized before using them
4. **Context Awareness**: Leverage thread history for more contextual conversations
5. **Resource Monitoring**: Monitor thread usage to avoid unnecessary resource consumption

## Use Cases

- **Customer Support**: Maintaining conversation history across support sessions
- **Multi-Session Chats**: Continuing conversations across different user sessions
- **Conversation Branching**: Managing multiple conversation topics simultaneously
- **Context Preservation**: Keeping conversation context for better user experience
- **Session Management**: Managing different conversation sessions for different users